# Series 8: Neural Networks feat. TensorFlow

## Problem 1 (Getting Started with TensorFlow)

Based on the code skeleton provided by our kind TAs.
Note: TF is not designed to handle multiple pipelines in a single
file without proper modularization. Because of this, it's indicated
to just run one of the problems at a time, since otherwise you might
stumble upon some weird errors.

In [6]:
import tensorflow as tf
import numpy as np
import datetime

### 1.1 Multi-valued linear regression

In [2]:
# create some noisy data
TRAIN_EXAMPLES = 100000
INPUT_DIMENSION = 2
OUTPUT_DIMENSION = 3
x_data = np.random.rand(TRAIN_EXAMPLES, INPUT_DIMENSION).astype(np.float32)

# correct_W = [[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12], [13, 14, 15]]
correct_W = [[1, 2, 3], [4, 5, 6]]
correct_b = [11, 12, 13]
correct_W, correct_b = map(lambda l: np.array(l, dtype=np.float32), (correct_W, correct_b))

noise_level = 0.01
y_data = np.dot(x_data, correct_W) + correct_b + np.random.normal(size=(TRAIN_EXAMPLES, OUTPUT_DIMENSION))

In [3]:
# It's better to work with a single large cell, in order
# to avoid stale tensors and stuff.

def problem_11():

    # define the symbolic variables
    W = tf.Variable(tf.random_uniform(correct_W.shape, -1.0, 1.0))
    b = tf.Variable(tf.zeros(correct_b.shape))

    # define the linear model
    # y_hat is symbolic!
    # 'x_data' is fixed (our data, duh!).
    y_hat = tf.matmul(x_data, W) + b

    # define the loss
    loss = tf.reduce_mean(tf.square(y_hat - y_data))
    tf.scalar_summary('log loss', tf.log(1.0 + loss))

    # define the optimizer
    step_size = 0.1
    optimizer = tf.train.GradientDescentOptimizer(step_size)
    train_op = optimizer.minimize(loss)

    # initialize the tensorflow session
    init = tf.initialize_all_variables()
    iterations = 500

    with tf.Session() as sess:
        sess.run(init)

        summary_op = tf.merge_all_summaries()
        summary_writer = tf.train.SummaryWriter("train/ex1_{}".format(datetime.datetime.now().strftime("%s")), sess.graph)

        # call the train_op many times, each time it will update the variables W and b according to their gradients
        for step in range(1, iterations + 1):
            _, loss_value, summary_str = sess.run([train_op, loss, summary_op])
            summary_writer.add_summary(summary_str, step)
            if step % 100 == 0:
                print("iteration:", step, "loss:", loss_value)

        print("learned W:\n{}".format(sess.run(W)))
        print("learned b:\n{}".format(sess.run(b)))

### 1.2 Batching and TensorBoard

In [4]:
def problem_12():
    # define the symbolic variables
    W = tf.Variable(tf.random_uniform(correct_W.shape, -1.0, 1.0))
    b = tf.Variable(tf.zeros(correct_b.shape))
    
    # define the data placeholders
    batch_size = 10
    x_ph = tf.placeholder(tf.float32, shape=(batch_size, INPUT_DIMENSION))
    y_ph = tf.placeholder(tf.float32, shape=(batch_size, OUTPUT_DIMENSION))

    # define the model (using placeholders)
    y_hat_batch = tf.matmul(x_ph, W) + b

    # define the (stochastic!) loss
    loss_batch = tf.reduce_mean(tf.square(y_hat_batch - y_ph))
    tf.scalar_summary('log loss', tf.log(1.0 + loss_batch))  # attention: this is the stochastic loss, i.e. it will be noisy

    # define the optimizer
    step_size = 0.1
    optimizer = tf.train.GradientDescentOptimizer(step_size)
    train_op = optimizer.minimize(loss_batch)

    # initialize the tensorflow session
    init = tf.initialize_all_variables()
    with tf.Session() as sess:
        sess.run(init)

        summary_op = tf.merge_all_summaries()
        summary_writer = tf.train.SummaryWriter("train/ex2_{}".format(datetime.datetime.now().strftime("%s")), sess.graph)

    # call the train_op many times, each time it will update the variables W and b according to their gradients
        for step in range(201):
            # determine the minibatch
            start_index = (batch_size * step) % x_data.shape[0]
            stop_index = start_index + batch_size

            # get the minibatch data
            x_minibatch = x_data[start_index:stop_index]
            y_minibatch = y_data[start_index:stop_index]

            feed_dict = {
                    x_ph: x_minibatch,
                    y_ph: y_minibatch
                    }

            _, loss_value, summary_str = sess.run([train_op, loss_batch, summary_op], feed_dict=feed_dict)
            summary_writer.add_summary(summary_str, step)
            print("iteration:", step, "loss:", loss_value)

        print("learned W:\n{}".format(sess.run(W)))
        print("learned b:\n{}".format(sess.run(b)))

In [5]:
problem_12()

iteration: 0 loss: 256.021
iteration: 1 loss: 205.065
iteration: 2 loss: 155.628
iteration: 3 loss: 136.074
iteration: 4 loss: 113.611
iteration: 5 loss: 87.1557
iteration: 6 loss: 71.481
iteration: 7 loss: 49.5685
iteration: 8 loss: 49.4222
iteration: 9 loss: 37.0111
iteration: 10 loss: 30.8024
iteration: 11 loss: 23.6687
iteration: 12 loss: 18.7336
iteration: 13 loss: 16.8241
iteration: 14 loss: 16.3116
iteration: 15 loss: 14.3766
iteration: 16 loss: 12.3003
iteration: 17 loss: 7.79278
iteration: 18 loss: 7.31078
iteration: 19 loss: 7.60889
iteration: 20 loss: 6.23099
iteration: 21 loss: 4.25003
iteration: 22 loss: 2.84943
iteration: 23 loss: 2.57385
iteration: 24 loss: 2.33063
iteration: 25 loss: 3.71958
iteration: 26 loss: 3.6538
iteration: 27 loss: 2.03445
iteration: 28 loss: 2.46061
iteration: 29 loss: 3.12589
iteration: 30 loss: 1.49019
iteration: 31 loss: 1.86478
iteration: 32 loss: 2.24545
iteration: 33 loss: 1.40993
iteration: 34 loss: 2.17417
iteration: 35 loss: 2.8454
itera